In [59]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from sheets import *

In [60]:
import mysql.connector

In [61]:
fixed_income = get_fixed_income()

In [62]:
variable_income = get_stock()

In [63]:
fgts = get_fgts()

In [64]:
cdi = get_cdi()

In [65]:
ipca = get_ipca()

In [66]:
target = get_target()

In [67]:
host = "finance.cvm2aaxkmu43.us-east-2.rds.amazonaws.com"
user = "admin"
password = "Nk7f4mJr6?A"
database = "finance"

conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = conn.cursor()

In [68]:
cursor.execute("DELETE FROM variable_income_operations")

In [69]:
cursor.executemany(
    """
    INSERT IGNORE INTO variable_income_operations(ticker, operation_type, operation_date, amount, price, currency)
    VALUES (%s, %s, %s, %s, %s, %s)
    """,
    variable_income
)

In [70]:
cursor.executemany(
    """
    INSERT IGNORE INTO index_series(financial_index, date, factor)
    VALUES (%s, %s, %s)
    """,
    cdi
)

In [71]:
cursor.executemany(
    """
    INSERT IGNORE INTO index_series(financial_index, date, factor)
    VALUES (%s, %s, %s)
    """,
    ipca
)

In [72]:
cursor.execute("DELETE FROM fixed_income_operations")

In [73]:
cursor.executemany(
    """
    INSERT IGNORE INTO fixed_income_operations (asset, operation_type, quotas, purchase_date, due_date, financial_index, value, pre_rate, post_rate, tax_rate, is_pgbl)
    VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """,
    fixed_income
)

In [74]:
cursor.execute("DELETE FROM fgts_operations")

In [75]:
cursor.executemany(
    """
    INSERT IGNORE INTO fgts_operations (date, company, operation, value, balance)
    VALUES(%s, %s, %s, %s, %s)
    """,
    fgts
)

In [76]:
cursor.execute("DELETE FROM target_percentage")

In [77]:
cursor.executemany(
    """
    INSERT IGNORE INTO target_percentage (name, percentage)
    VALUES(%s, %s)
    """,
    target
)

In [78]:
conn.commit()
print("Upserted data successfully.")

Upserted data successfully.


In [79]:
conn.close()